# Exam 2: Eduardo Abenza Severá

Before starting these exercises, I have executed this line of code in a Linux terminal window:

> sudo docker start course-mysql

## Problem 1: Controls

- Write a Python script that proves that the lines of data in Germplasm.tsv, and LocusGene.tsv are in the same sequence, based on the AGI Locus Code (ATxGxxxxxx). (hint: This will help you decide how to load the data into the database)

### Answer 1

First, I will take a look at both *.tsv* files. In this first block of code, I'm opening **Germplasm.tsv** as **germplasm** and I am printing its content (first 1000 characters):

In [ ]:
germplasm = open("Germplasm.tsv", "r")
print(germplasm.read()[:500])

This file is compound by four columns, called **Locus**, **germplasm**, **phenotype** and **pubmed**. These columns, and the values in each row, are delimited by tabs. In this exercise, we are interested in the values before the first tab, because they are the AGI Locus codes.

Now, we are exploring the content (first 198 characters) in the second file, **LocusGene.tsv**, through an object called **locus**:

In [ ]:
locus = open("LocusGene.tsv", "r")
print(locus.read()[:100])

This file is compound by three columns, called **Locus**, **Gene**, **phenotype** and **pubmed**. These column names, and the values in each row, are delimited by tabs. In this exercise, also we are interested in the values before the first tab in this file, because they are the AGI Locus codes.

In the next cell I'm creating **germ** and **loc** lists, which contains all lines of **germplasm** and **locus**. Each line constitutes an element in the list.

In [ ]:
germplasm.seek(0)
locus.seek(0)

germ = germplasm.readlines()
loc = locus.readlines()

print("\nFirst four lines in Germplasm.tsv:\n\n" + str(germ[:4]) + "\n\nNumber of rows: " + str(len(germ)))
print("\n\n\nFirst four lines in LocusGene.tsv:\n\n"+ str(loc[:4]) + "\n\nNumber of rows: " + str(len(loc)))

Both files have 33 lines, and the first ones are headers of columns.

The aim of this exercise was to determine whether the lines of these files are in the same sequence, based on the AGI Locus code. In the next piece of code I'm extracting AGI Locus codes of both files in each line, and comparing them to determine whether both AGI Locus codes are equal or not.

In [ ]:
import re

for _ in range(1, 33):
    mog = re.search('([^\t]+)', germ[_])
    mol = re.search('([^\t]+)', loc[_])
    g, l = mog.group(1), mol.group(1)
    print("Row number %i\nGermplasm: %s, LocusGene: %s\nAre they equal? %s" % (_, g, l, g == l))
    if not g==l:
        print("THESE AGI LOCUS CODES ARE NOT EQUAL!")
    print("")

All couples of AGI Locus code are equal.

## Problem 2:  Design and create the database.  
* It should have two tables - one for each of the two data files.
* The two tables should be linked in a 1:1 relationship
* you may use either sqlMagic or pymysql to build the database

### Answer 2

The first step is the design of the two new tables. The new **germplasm** table, as Germplasm.tsv, will have four columns and their headers will be **Locus**, **germplasm**, **phenotype** and **pubmed**. I have decided to use AGI Locus code as primary key for each row. It will look like this:

<center><strong>germplasm</strong></center>

   Locus  |  germplasm  |  phenotype  |  pubmed  
  --- | --- | --- | ---
  AT1G_ _ _ _ _ | germplasm1 |  description1  |  number1
  AT_G_ _ _ _ _  | germplasm2 |  description2  | number2 
  AT_G_ _ _ _ _  |  germplasm3  |  description3  |  number3


-----------------------------

On the other hand, the new **locusgene** table, as LocusGene.tsv, will have three columns and their headers will be **Locus**, **gene** and **ProteinLength**. Also, I have decided to use AGI Locus code as primary key for each row. It will look like this:

<center><strong>locusgene</strong></center>

   Locus  |  gene  |  ProteinLength  
  --- | --- | --- 
   AT1G_ _ _ _ _ | gene1 |  length1
  AT_G_ _ _ _ _  | gene2 | length2 
   AT_G_ _ _ _ _  |  gene3  |  length3


-----------------------------

With respect to the 1:1 relationship between **germplasm** and **locusgene** tables, I showed in **Question 1** that all AGI Locus codes appear in the same sequence in both files. These two tables may be linked by the AGI Locus codes.

### Creation of exam2 database

Now, I'm creating a new database. After loading **cursors** from **pymysql** package, I'm beginning this exercise with the creation of a new SQL database called **Exam 2**.

In [ ]:
import pymysql.cursors

# Connect to the database
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='root',
                             charset='utf8mb4',  # Important for unusual characters
                             cursorclass=pymysql.cursors.DictCursor)


try:
    with connection.cursor() as cursor:
        sql = "CREATE DATABASE exam2"
        cursor.execute(sql)
        sql = "SHOW DATABASES"
        cursor.execute(sql)
        print(cursor.fetchall())
finally:
    print("")

Next, I'm telling SQL I'm using the new database. Initially, it is empty:

In [ ]:
try:
    with connection.cursor() as cursor:
        sql = "USE exam2"
        cursor.execute(sql)
        sql = "SHOW TABLES"
        cursor.execute(sql)
        print(cursor.fetchall())
finally:
    print("")

### Creation of germplasm table

To perform this task, I'm splitting (by \t) the column names in the first line of **germ** and I'm storing them in a list called **gcolnames**. Then, I'm creating a SQL command by concatenating all items in the list (i.e., the column names) with the characteristics of each column in an appropriate manner.

In [ ]:
try:
    with connection.cursor() as cursor:
        gcolnames = germ[0].split("\t") 
        sql = """CREATE TABLE germplasm( {} VARCHAR(9) NOT NULL 
        PRIMARY KEY, {} VARCHAR(25) NOT NULL, {} VARCHAR(500) NOT NULL, 
        {} INTEGER NOT NULL)""".format(gcolnames[0],gcolnames[1], gcolnames[2], gcolnames[3])
        cursor.execute(sql)
        sql = "DESCRIBE germplasm"
        cursor.execute(sql)
        for line in cursor.fetchall():
            print(line)
finally:
    print("")

With this block of code, we get a table with the desired characteristics.

### Creation of locusgene table

I'm repeating the same procedure I followed to create the **germplasm** table. I'm storing the column names of **loc** object in a new list (**lcolnames**) and I'm concatenating this column names with SQL commands in a suitable manner.

In [ ]:
try:
    with connection.cursor() as cursor:
        lcolnames = loc[0].split("\t") 
        sql = """CREATE TABLE locusgene( {} VARCHAR(9) NOT NULL 
        PRIMARY KEY, {} VARCHAR(25) NOT NULL, 
        {} INTEGER NOT NULL)""".format(lcolnames[0],lcolnames[1], lcolnames[2])
        cursor.execute(sql)
        sql = "DESCRIBE locusgene"
        cursor.execute(sql)
        for line in cursor.fetchall():
            print(line)
finally:
    print("")

This second table is also correct.

## Problem 3: Fill the database
- Using pymysql, create a Python script that reads the data from these files, and fills the database.  There are a variety of strategies to accomplish this.  I will give all strategies equal credit - do whichever one you are most confident with.

### Answer 3

To accomplish the objective of this exercise, I am defining a new function. The input of this function, called **sql_insert_into**, are the name of the table, the name of the original file where the values are located and the delimiter used to separate values in the original file. This simple function outputs a list with all SQL **INSERT INTO** commands that are necessary to fill the table with the data of the file.

In the cell below I am defining this new function.

In [ ]:
def sql_insert_into(table, file, delimiter):
    """
    This function reads the name of the table where you want to insert the values (table, string),
    the name of the file where the values and column names are found (file, string) and the delimiter
    which separates values in the same row in the file. 
    At the end, it returns a list with all SQL commands you need to execute to fill the table.
    """
    file = open(file, "r")
    file_lines = file.readlines()
    colnames = file_lines[0].replace("\n", "").split(delimiter)
    sql_commands = [] # Empty list, which will be filled and returned when this function reaches its end.
    for _ in range(1, len(file_lines)):
        values = file_lines[_].replace("\n", "").split(delimiter)  # It takes the string in line number _,
        # removes the final \n and splits the list into a sublist of values, previously separated by delimiter.
        sql_line_command = "INSERT INTO {} ( {} ) VALUES ( '{}' );".format(\
            table, ", ".join(colnames), "', '".join(values))
        sql_commands.append(sql_line_command) # Appends the command to introduce values in this row to 
        # sql_commands list
    file.close()
    return sql_commands

Once defined this function, we can fill both tables with the values found in the original files.

### Filling of germplasm table

We give **sql_insert_into** function the required strings, and execute all SQL commands in the returned list, one by one. Finally, we check the introduced values by selecting all values in this table:

In [ ]:
try:
    with connection.cursor() as cursor:
        for sql in sql_insert_into("germplasm", "Germplasm.tsv", "\t"):
            cursor.execute(sql)
        cursor.execute("SELECT * FROM germplasm")
        for line in cursor.fetchall()[0:3]:
            print(line)
finally:
    print("")

It seems that all values have been added correctly to this table.

### Filling of locusgene table

Now, we repeat the same procedure as in the previous line of code with **locusgene** table.


In [ ]:
try:
    with connection.cursor() as cursor:
        for sql in sql_insert_into("locusgene", "LocusGene.tsv", "\t"):
            cursor.execute(sql)
        cursor.execute("SELECT * FROM locusgene")
        for line in cursor.fetchall()[0:3]:
            print(line)
finally:
    print("")

This table also looks nice!

## Problem 4: Create reports, written to a file

1. Create a report that shows the full, joined, content of the two database tables (including a header line)

2. Create a joined report that only includes the Genes SKOR and MAA3

3. Create a report that counts the number of entries for each Chromosome (AT1Gxxxxxx to AT5Gxxxxxxx)

4. Create a report that shows the average protein length for the genes on each Chromosome (AT1Gxxxxxx to AT5Gxxxxxxx)

When creating reports 2 and 3, remember the "Don't Repeat Yourself" rule! 

All reports should be written to **the same file**.  You may name the file anything you wish.

### Answer 4

In this exercise, I am creating a report that shows different operations on **exam2** database tables. To write the final tsv file, I am defining a new function. This function outputs a string with multiple lines of tab-separated values (abbreviated: *tsv-string*), after reading a DictCursor dictionary (from *pymysql*) and a previous tsv-string (if existed). At the end of this exercise, this tsv-string will be written into a tsv file to show the four operations performed in this exercise.

In the next piece of code I am defining this function.

In [ ]:
def pymysql_to_tsv(dictCursor, tsv_string = ""):
    colnames = list(dictCursor[0].keys())
    tsv_string = "\t".join(colnames) + "\n" # This line of code appends
    # the header to a new line in tsv_string
    for row in dictCursor:
        values = []   # A new list for each row
        for colname in colnames:
            values.append(str(row[colname]))   # Values are appended to "values" list as strings
        tsv_string = tsv_string + "\t".join(values) + "\n"  # Each row is concatenated to the previous
        # tsv_string separated by \n, with their values separated by tabs.
    return tsv_string + "\n\n"   # The function returns a tsv_string, isolated from the following 
    # tsv_strings by two new line symbols.

#### Full, joined content of the two database tables (including a header line)

I am writing the SQL command to full join both tables by locus column. We checked in **Question 1** that there are the same AGI Locus codes in both files, so any kind of join (inner, right, left and outer join) is suitable. I am using the first one, inner join.

Then, using **pymysql_to_tsv** defined in the previous block of code, we write the output of this SQL command into a string. This string will be written to a file at the end of **Question 4**.

In [ ]:
try:
    with connection.cursor() as cursor:
        sql = "SELECT * FROM locusgene INNER JOIN germplasm ON germplasm.locus = locusgene.locus;"
        cursor.execute(sql)
        tsv_string = pymysql_to_tsv(cursor.fetchall())
        comment_1 = "# This table contains full joined content of the two database tables, called germplasm and locusgene.\n\n"
        report = comment_1 + tsv_string
        print(report[:400])
                
finally:
    print("")

It seems that the **report** string we have generated is correct.

#### Joined report that only includes the Genes SKOR and MAA3

To achieve this, we repeat the same SQL command used in the previous question with a slight modification: it only includes the rows where the gene is SKOR or MAA3.

In [ ]:
try:
    with connection.cursor() as cursor:
        sql = "SELECT * FROM locusgene INNER JOIN germplasm ON germplasm.locus = locusgene.locus WHERE gene IN ('SKOR', 'MAA3');"
        cursor.execute(sql)
        tsv_string = pymysql_to_tsv(cursor.fetchall()) 
        comment_2 = "# This second table contains full joined content of germplasm and locusgene tables, but only for rows with genes SKOR and MAA3.\n\n"
        report = report + comment_2 + tsv_string
        print(report[-600:])
                
finally:
    print("")


Taking a look at the last six hundreds of characters in the **report** string, we see that it looks nice.

#### Report with count of the number of entries for each Chromosome

In this section, I think it is advisable to create a new table in the database. This new table is equal to **locusgene** table, with an extra column: this new column shows the chromosome number for each entry.

We define a new function, slightly modifying **sql_insert_into**:

- In the column names (**colnames**) list, it includes a new name: **Chromosome**.
- In the values list (**values**), we find a new element: the character in position number 3 of AGI Locus code, i.e., the **chromosome number**.

In [ ]:
def sql_chromosome(table, file, delimiter):
    """
    This function reads the name of the table where you want to insert the values (table, string),
    the name of the file where the values and column names are found (file, string) and the delimiter
    which separates values in the same row in the file. 
    At the end, it returns a list with all SQL commands you need to execute to fill the table.
    (Same code as sql_insert_into function, with small modifications to create chromosome table.)
    """
    file = open(file, "r")
    file_lines = file.readlines()
    colnames = file_lines[0].replace("\n", "").split(delimiter)
    colnames.append("Chromosome")
    sql_commands = [] # Empty list, which will be filled and returned when this function reaches its end.
    for _ in range(1, len(file_lines)):
        values = file_lines[_].replace("\n", "").split(delimiter)  # It takes the string in line number _,
        # removes the final \n and splits the list into a sublist of values, previously separated by delimiter.
        values.append(values[0][2])
        sql_line_command = "INSERT INTO {} ( {} ) VALUES ( '{}' );".format(\
            table, ", ".join(colnames), "', '".join(values))
        sql_commands.append(sql_line_command) # Appends the command to introduce values in this row to 
        # sql_commands list
    file.close()
    return sql_commands

After defining this new function, we create the new table. Chromosome column only admits integer numbers.

In [ ]:
try:
    with connection.cursor() as cursor:
        lcolnames = loc[0].split("\t")
        sql = """CREATE TABLE chromosome( {} VARCHAR(9) NOT NULL 
        PRIMARY KEY, {} VARCHAR(25) NOT NULL, {} INTEGER NOT NULL, 
        Chromosome INTEGER NOT NULL)""".format(lcolnames[0],lcolnames[1], lcolnames[2])
        cursor.execute(sql)
        sql = "DESCRIBE chromosome"
        cursor.execute(sql)
        for line in cursor.fetchall():
            print(line)
finally:
    print("")

The structure of this table is correct.

Now, we make use of **sql_chromosome** function to fill this new table.

In [ ]:
try:
    with connection.cursor() as cursor:
        for sql in sql_chromosome("chromosome", "LocusGene.tsv", "\t"):
            cursor.execute(sql)
        cursor.execute("SELECT * FROM chromosome")
        for line in cursor.fetchall()[:3]:
            print(line)
finally:
    print("")

The new function has worked properly, and the new table is already filled.

Now, we can create a new table which contains two columns (number of chromosome and number of entries for that chromosome), by grouping by chromosome number and selecting the **Chromosome** column and the count of rows for each chromosome after the grouping.

In [ ]:
try:
    with connection.cursor() as cursor:
        sql = "SELECT Chromosome, count(*) AS Occurences FROM chromosome GROUP BY Chromosome;"
        cursor.execute(sql)
        tsv_string = pymysql_to_tsv(cursor.fetchall())
        comment_3 = "# This table shows the count of entries in the original files for each chromosome. " + \
        "To perform this task, I have created a new table with the number of chromosome for every row, " + \
        "and I have grouped by chromosome number in order to count the number of entries.\n\n"
        report = report + comment_3 + tsv_string
        print(report[-350:])
                
finally:
    print("")

The output of this piece of code appears to be what we wanted. There are only four entries for chromosomes number 1 and 2, and chromosome number 3 is the one with the highest number of entries (9).

#### Report showing the average protein length for the genes on each chromosome 

Making use of **chromosome** table, we can obtain the average protein length for the genes on each chromosome. We group by **Chromosome** and output a table with two columns: the **Chromosome** column itself and the average protein length for genes on each chromosome.

In [ ]:
try:
    with connection.cursor() as cursor:
        sql = "SELECT Chromosome, AVG(ProteinLength) AS 'AverageProteinLength' FROM chromosome GROUP BY Chromosome;"
        cursor.execute(sql)
        tsv_string = pymysql_to_tsv(cursor.fetchall())   
        comment_4 = "# This last table contains the average protein length for genes on each chromosome. To " + \
        "obtain this table, I have grouped the rows in chromosome table by chromosome number and I have " + \
        "computed the average protein length on each chromosome.\n\n"
        report = report + comment_4 + tsv_string
        print(report[-350:])
                
finally:
    print("")

We have finished the operations on tables of **exam2** database. Now, we can delete this database and write the **tsv_string** with all tables to a new file called **report.txt**. This file contains all tables, with values separated by tabs, and a little comment above all tables to explain what I have done to obtain them.

In [ ]:
try:
    with connection.cursor() as cursor:
        cursor.execute("DROP DATABASE exam2")
                
finally:
    print("")
    connection.close()

f = open("report.txt", "w")
f.write(report)
f.close()